In [1]:
# Change the dataset in global_vars.py to NAKO.
from global_vars import *
from commons import *

import glob 
import os

one_time_n4_optimization = True
vol_to_check_list = None #['100025']
exclude = ['100006']  # No SPleen
# 181
%load_ext autoreload
%autoreload 2

In [2]:
def load_nako_file_paths(load_from_txt_file=True):
    volumes_to_use = []
    if load_from_txt_file:
        with open(volume_txt_file) as file_handle:
                volumes_to_use = file_handle.read().splitlines()
    else:
        volumes_to_use = [name for name in os.listdir(data_dir)]
    
    file_paths = {}
    
    for vol in volumes_to_use:
        if (vol_to_check_list is not None and vol not in vol_to_check_list) or (vol == "") or (vol in exclude):
            continue
        opp_paths = glob.glob(f'{data_dir}/{vol}/{vol}_3D_GRE_TRA_opp/**.nii.gz') # **_2**
        in_paths = glob.glob(f'{data_dir}/{vol}/{vol}_3D_GRE_TRA_in/**.nii.gz')
        f_paths = glob.glob(f'{data_dir}/{vol}/{vol}_3D_GRE_TRA_F/**.nii.gz')
        w_paths = glob.glob(f'{data_dir}/{vol}/{vol}_3D_GRE_TRA_W/**.nii.gz')
        
        labelmap_paths = glob.glob(f'{label_dir}/{vol}/**')
        
        vol_madals_paths = dict(
        OPP=opp_paths,
        IN=in_paths,
        F=f_paths,
        W=w_paths
        )
        file_paths[str(vol)]=dict(
            VOLUME_PATHS=vol_madals_paths,
            LABEL_PATHS=labelmap_paths,
        )
    return file_paths

file_paths = load_nako_file_paths()
file_paths

{'100000': {'VOLUME_PATHS': {'OPP': ['/mnt/nas/Data_WholeBody/NAKO/NAKO_200/MRI/100000/100000_3D_GRE_TRA_opp/3D_GRE_TRA_opp_3D_GRE_TRA_1.nii.gz',
    '/mnt/nas/Data_WholeBody/NAKO/NAKO_200/MRI/100000/100000_3D_GRE_TRA_opp/3D_GRE_TRA_opp_3D_GRE_TRA_2.nii.gz',
    '/mnt/nas/Data_WholeBody/NAKO/NAKO_200/MRI/100000/100000_3D_GRE_TRA_opp/3D_GRE_TRA_opp_3D_GRE_TRA_3.nii.gz',
    '/mnt/nas/Data_WholeBody/NAKO/NAKO_200/MRI/100000/100000_3D_GRE_TRA_opp/3D_GRE_TRA_opp_3D_GRE_TRA_4.nii.gz'],
   'IN': ['/mnt/nas/Data_WholeBody/NAKO/NAKO_200/MRI/100000/100000_3D_GRE_TRA_in/3D_GRE_TRA_in_3D_GRE_TRA_1_e2.nii.gz',
    '/mnt/nas/Data_WholeBody/NAKO/NAKO_200/MRI/100000/100000_3D_GRE_TRA_in/3D_GRE_TRA_in_3D_GRE_TRA_2_e2.nii.gz',
    '/mnt/nas/Data_WholeBody/NAKO/NAKO_200/MRI/100000/100000_3D_GRE_TRA_in/3D_GRE_TRA_in_3D_GRE_TRA_3_e2.nii.gz',
    '/mnt/nas/Data_WholeBody/NAKO/NAKO_200/MRI/100000/100000_3D_GRE_TRA_in/3D_GRE_TRA_in_3D_GRE_TRA_4_e2.nii.gz'],
   'F': ['/mnt/nas/Data_WholeBody/NAKO/NAKO_200/MRI

In [3]:
# Individual RESCALING.
n4_dict = {}
for vol in file_paths.keys():
    if one_time_n4_optimization:
        break
    if vol_to_check_list is not None and vol not in vol_to_check_list:
        continue
    print(f'n4 processing part-1 started with {vol}...')
    n4_dict[vol] = []
    vol_parts = [[file, read_ras(file)] for file in file_paths[vol]['VOLUME_PATHS']['IN']]
    for orig_file, in_image in vol_parts:
        n4_dict[vol].append(rescale(in_image, vol, orig_file))

In [4]:
# SITK does ot work due to differences in pixel resolution of IN and corresponding OPP Scan.
# Only applying once at the end.
for vol in file_paths.keys():
    if one_time_n4_optimization:
        break
    if vol_to_check_list is not None and vol not in vol_to_check_list:
        continue
    print(f'n4-biasfield-correction starting with {vol}...')
    for idx, n4_d in enumerate(n4_dict[vol]):
        in_file = n4_d['SCALED']
        opp_file = file_paths[vol]['VOLUME_PATHS']['OPP'][idx]
        new_filename = opp_file.split('/')[-1].split('.')[0]
        output_file = f'{n4_corrected_data_dir}/vol/{vol}/{new_filename}_n4_corrected_sitk.nii.gz'
        SITK_N4_normalization(in_file, opp_file, output_file)
        n4_dict[vol][idx]['OPP_CORRECTED'] = output_file

    file_paths[vol]['N4_1'] = n4_dict[vol]

In [5]:
# STITCHING VOL PARTS HERE
for vol in file_paths.keys():
    try:
        print(f'started with {vol}...')
        if vol_to_check_list is not None and vol not in vol_to_check_list:
            continue
        create_if_not(f'{n4_corrected_data_dir}/vol/{vol}')
        file_paths[vol]['ONE'] = {}
        for modality_key in file_paths[vol]['VOLUME_PATHS'].keys():
            print(f"processing {modality_key}")
            orig_modal_key = modality_key
            if one_time_n4_optimization:
                vol_parts = [read_ras(file) for file in file_paths[vol]['VOLUME_PATHS'][modality_key]]
            else:
                if modality_key == 'OPP':
                    vol_parts = [read_ras(data_dict['OPP_CORRECTED']) for data_dict in file_paths[vol]['N4_1']]
                    modality_key = modality_key+'_n4_corrected'
                else:
                    vol_parts = [read_ras(file) for file in file_paths[vol]['VOLUME_PATHS'][modality_key]]

            ras_stitched = multi_vol_stitching(vol_parts)
            save_volume(ras_stitched, f'{n4_corrected_data_dir}/vol/{vol}/{modality_key}_ras_stitched')
            file_paths[vol]['ONE'][f'{orig_modal_key}'] = f'{n4_corrected_data_dir}/vol/{vol}/{modality_key}_ras_stitched.nii.gz'
    except Exception as e:
        print('ERROR:',e)
        continue

started with 100000...
processing OPP
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -233.59375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.      -407.5    ]
 [   0.         0.         0.         1.     ]], Image Shape: (320, 260, 96)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -233.59375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.      -599.5    ]
 [   0.         0.         0.         1.     ]], Image Shape: (320, 260, 96)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -233.59375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.      -827.5    ]
 [   0.         0.         0.         1.     ]], Image Shape: (320, 260, 88)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[ 1.406

Transforming Images to RAS.....
0th img for stitching...
1th img for stitching...
2th img for stitching...
saving directory: /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp/NAKO/n4_corrected_2/vol/100001
processing F
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -223.59375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.      -470.     ]
 [   0.         0.         0.         1.     ]], Image Shape: (320, 260, 96)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -223.59375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.      -662.     ]
 [   0.         0.         0.         1.     ]], Image Shape: (320, 260, 96)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -223.59375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.

Transforming Images to RAS.....
0th img for stitching...
1th img for stitching...
2th img for stitching...
saving directory: /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp/NAKO/n4_corrected_2/vol/100002
started with 100003...
processing OPP
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -236.09375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.      -417.5    ]
 [   0.         0.         0.         1.     ]], Image Shape: (320, 260, 96)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -236.09375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.      -609.5    ]
 [   0.         0.         0.         1.     ]], Image Shape: (320, 260, 96)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -236.09375]
 [   0.         1.40625    0.      -178.90625]
 [  

Transforming Images to RAS.....
0th img for stitching...
1th img for stitching...
2th img for stitching...
saving directory: /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp/NAKO/n4_corrected_2/vol/100004
processing F
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -221.09375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.      -475.     ]
 [   0.         0.         0.         1.     ]], Image Shape: (320, 260, 96)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -221.09375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.      -667.     ]
 [   0.         0.         0.         1.     ]], Image Shape: (320, 260, 96)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -221.09375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.

0th img for stitching...
1th img for stitching...
2th img for stitching...
saving directory: /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp/NAKO/n4_corrected_2/vol/100005
started with 100007...
processing OPP
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -233.59375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.      -385.     ]
 [   0.         0.         0.         1.     ]], Image Shape: (320, 260, 96)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -233.59375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.      -577.     ]
 [   0.         0.         0.         1.     ]], Image Shape: (320, 260, 96)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -233.59375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.      -

...

Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -216.09375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.      -845.     ]
 [   0.         0.         0.         1.     ]], Image Shape: (320, 260, 88)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[ 1.4062500e+00  0.0000000e+00  0.0000000e+00 -2.1609375e+02]
 [ 0.0000000e+00  1.4062500e+00  0.0000000e+00 -1.7890625e+02]
 [ 0.0000000e+00  0.0000000e+00  3.0000000e+00 -1.0850000e+03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]], Image Shape: (320, 260, 88)
Transforming Images to RAS.....
0th img for stitching...
1th img for stitching...
2th img for stitching...
saving directory: /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp/NAKO/n4_corrected_2/vol/100194
processing F
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -216.09375]
 [   0.     

processing W
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -241.09375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.      -425.     ]
 [   0.         0.         0.         1.     ]], Image Shape: (320, 260, 96)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -241.09375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.      -617.     ]
 [   0.         0.         0.         1.     ]], Image Shape: (320, 260, 96)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -241.09375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.      -845.     ]
 [   0.         0.         0.         1.     ]], Image Shape: (320, 260, 88)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[ 1.4062500e+00  0.0000000e+00  

Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[ 1.4062500e+00  0.0000000e+00  0.0000000e+00 -2.1359375e+02]
 [ 0.0000000e+00  1.4062500e+00  0.0000000e+00 -1.7890625e+02]
 [ 0.0000000e+00  0.0000000e+00  3.0000000e+00 -1.0675000e+03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]], Image Shape: (320, 260, 88)
Transforming Images to RAS.....
0th img for stitching...
1th img for stitching...
2th img for stitching...
saving directory: /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp/NAKO/n4_corrected_2/vol/100197
processing IN
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -213.59375]
 [   0.         1.40625    0.      -178.90625]
 [   0.         0.         3.      -407.5    ]
 [   0.         0.         0.         1.     ]], Image Shape: (320, 260, 96)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625    0.         0.      -213.59375]
 [   0.    

processing F
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625       0.            0.         -234.5259552 ]
 [   0.            1.40625       0.         -178.90625   ]
 [   0.            0.            3.         -465.99575806]
 [   0.            0.            0.            1.        ]], Image Shape: (320, 260, 96)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625       0.            0.         -234.5259552 ]
 [   0.            1.40625       0.         -178.90625   ]
 [   0.            0.            3.         -657.99572754]
 [   0.            0.            0.            1.        ]], Image Shape: (320, 260, 96)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625       0.            0.         -234.5259552 ]
 [   0.            1.40625       0.         -178.90625   ]
 [   0.            0.            3.         -885.99578857]
 [   0.            0.            0.            1.        ]], 

Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625       0.            0.         -223.59375   ]
 [   0.            1.40625       0.         -192.70126343]
 [   0.            0.            3.         -599.70080566]
 [   0.            0.            0.            1.        ]], Image Shape: (320, 260, 96)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[   1.40625       0.            0.         -223.59375   ]
 [   0.            1.40625       0.         -192.70126343]
 [   0.            0.            3.         -827.70080566]
 [   0.            0.            0.            1.        ]], Image Shape: (320, 260, 88)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[ 1.40625000e+00  0.00000000e+00  0.00000000e+00 -2.23593750e+02]
 [ 0.00000000e+00  1.40625000e+00  0.00000000e+00 -1.92701263e+02]
 [ 0.00000000e+00  0.00000000e+00  3.00000000e+00 -1.06770081e+03]
 [ 0.00000000e+00 

In [6]:
# RESCALING INTENSITIES OF STITCHED VOLUME ABOVE 0
n4_dict = {}
for vol in file_paths.keys():
    try:
        if vol_to_check_list is not None and vol not in vol_to_check_list:
            continue
        print(f'n4 processing part-2 started with {vol}...')
        n4_dict[vol] = {}
        in_stitched_file_path, in_stitched_img = file_paths[vol]['ONE']['IN'], read_ras(file_paths[vol]['ONE']['IN'])
        n4_dict[vol]['N4_2'] = rescale(in_stitched_img, vol, in_stitched_file_path)
        
    except Exception as e:
        print('ERROR:',e)
        continue

n4 processing part-2 started with 100000...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.0000000e+00  0.0000000e+00  0.0000000e+00 -2.3359375e+02]
 [ 0.0000000e+00  2.0000000e+00  0.0000000e+00 -1.7890625e+02]
 [ 0.0000000e+00  0.0000000e+00  3.0000000e+00 -1.0675000e+03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
-183.99993896484375
neagtive value detected
saving directory: /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp/NAKO/n4_corrected_2/vol/100000
n4 processing part-2 started with 100001...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.0000000e+00  0.0000000e+00  0.0000000e+00 -2.2359375e+02]
 [ 0.0000000e+00  2.0000000e+00  0.0000000e+00 -1.7890625e+02]
 [ 0.0000000e+00  0.0000000e+00  3.0000000e+00 -1.1300000e+03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
-143.0
neagtive value detected
s

n4 processing part-2 started with 100021...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.00000000e+00  0.00000000e+00  0.00000000e+00 -2.45551193e+02]
 [ 0.00000000e+00  2.00000000e+00  0.00000000e+00 -1.78906250e+02]
 [ 0.00000000e+00  0.00000000e+00  3.00000000e+00 -1.08500000e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
-93.0
neagtive value detected
saving directory: /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp/NAKO/n4_corrected_2/vol/100021
n4 processing part-2 started with 100022...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.0000000e+00  0.0000000e+00  0.0000000e+00 -2.1609375e+02]
 [ 0.0000000e+00  2.0000000e+00  0.0000000e+00 -1.7890625e+02]
 [ 0.0000000e+00  0.0000000e+00  3.0000000e+00 -1.0800000e+03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
-141.0
neagtive value detected

n4 processing part-2 started with 100038...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.0000000e+00  0.0000000e+00  0.0000000e+00 -2.1609375e+02]
 [ 0.0000000e+00  2.0000000e+00  0.0000000e+00 -1.7890625e+02]
 [ 0.0000000e+00  0.0000000e+00  3.0000000e+00 -1.1150000e+03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
-114.0
neagtive value detected
saving directory: /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp/NAKO/n4_corrected_2/vol/100038
n4 processing part-2 started with 100039...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.00000000e+00  0.00000000e+00  0.00000000e+00 -2.32270874e+02]
 [ 0.00000000e+00  2.00000000e+00  0.00000000e+00 -1.78906250e+02]
 [ 0.00000000e+00  0.00000000e+00  3.00000000e+00 -1.06406958e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
-118.0
neagtive value detecte

n4 processing part-2 started with 100053...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.0000000e+00  0.0000000e+00  0.0000000e+00 -2.1859375e+02]
 [ 0.0000000e+00  2.0000000e+00  0.0000000e+00 -1.7890625e+02]
 [ 0.0000000e+00  0.0000000e+00  3.0000000e+00 -1.0550000e+03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
-139.0
neagtive value detected
saving directory: /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp/NAKO/n4_corrected_2/vol/100053
n4 processing part-2 started with 100054...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.0000000e+00  0.0000000e+00  0.0000000e+00 -2.2359375e+02]
 [ 0.0000000e+00  2.0000000e+00  0.0000000e+00 -1.7890625e+02]
 [ 0.0000000e+00  0.0000000e+00  3.0000000e+00 -1.0850000e+03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
-120.0
neagtive value detected
saving directo

n4 processing part-2 started with 100068...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.0000000e+00  0.0000000e+00  0.0000000e+00 -2.1859375e+02]
 [ 0.0000000e+00  2.0000000e+00  0.0000000e+00 -1.7890625e+02]
 [ 0.0000000e+00  0.0000000e+00  3.0000000e+00 -1.0750000e+03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
-104.0
neagtive value detected
saving directory: /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp/NAKO/n4_corrected_2/vol/100068
n4 processing part-2 started with 100069...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.00000000e+00  0.00000000e+00  0.00000000e+00 -2.20621613e+02]
 [ 0.00000000e+00  2.00000000e+00  0.00000000e+00 -1.93766937e+02]
 [ 0.00000000e+00  0.00000000e+00  3.00000000e+00 -1.09319922e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
-111.0
neagtive value detecte

...

n4 processing part-2 started with 100135...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.00000000e+00  0.00000000e+00  0.00000000e+00 -2.25931107e+02]
 [ 0.00000000e+00  2.00000000e+00  0.00000000e+00 -1.86406250e+02]
 [ 0.00000000e+00  0.00000000e+00  3.00000000e+00 -1.09902417e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
-144.0
neagtive value detected
saving directory: /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp/NAKO/n4_corrected_2/vol/100135
n4 processing part-2 started with 100136...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.0000000e+00  0.0000000e+00  0.0000000e+00 -2.1109375e+02]
 [ 0.0000000e+00  2.0000000e+00  0.0000000e+00 -2.0140625e+02]
 [ 0.0000000e+00  0.0000000e+00  3.0000000e+00 -1.0475000e+03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
-110.0
neagtive value detecte

n4 processing part-2 started with 100151...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.0000000e+00  0.0000000e+00  0.0000000e+00 -2.4109375e+02]
 [ 0.0000000e+00  2.0000000e+00  0.0000000e+00 -1.7890625e+02]
 [ 0.0000000e+00  0.0000000e+00  3.0000000e+00 -1.0625000e+03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
-139.00006103515625
neagtive value detected
saving directory: /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp/NAKO/n4_corrected_2/vol/100151
n4 processing part-2 started with 100152...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.0000000e+00  0.0000000e+00  0.0000000e+00 -2.1609375e+02]
 [ 0.0000000e+00  2.0000000e+00  0.0000000e+00 -1.7890625e+02]
 [ 0.0000000e+00  0.0000000e+00  3.0000000e+00 -1.1025000e+03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
-83.4271240234375
neagtive value

n4 processing part-2 started with 100167...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.00000000e+00  0.00000000e+00  0.00000000e+00 -2.45034195e+02]
 [ 0.00000000e+00  2.00000000e+00  0.00000000e+00 -1.78906250e+02]
 [ 0.00000000e+00  0.00000000e+00  3.00000000e+00 -1.06355957e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
-93.0
neagtive value detected
saving directory: /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp/NAKO/n4_corrected_2/vol/100167
n4 processing part-2 started with 100168...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.0000000e+00  0.0000000e+00  0.0000000e+00 -2.1859375e+02]
 [ 0.0000000e+00  2.0000000e+00  0.0000000e+00 -1.7890625e+02]
 [ 0.0000000e+00  0.0000000e+00  3.0000000e+00 -1.0425000e+03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
-125.0
neagtive value detected

n4 processing part-2 started with 100184...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.00000000e+00  0.00000000e+00  0.00000000e+00 -2.34313965e+02]
 [ 0.00000000e+00  2.00000000e+00  0.00000000e+00 -1.62507950e+02]
 [ 0.00000000e+00  0.00000000e+00  3.00000000e+00 -1.11358728e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
-131.0
neagtive value detected
saving directory: /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp/NAKO/n4_corrected_2/vol/100184
n4 processing part-2 started with 100185...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.0000000e+00  0.0000000e+00  0.0000000e+00 -2.2502034e+02]
 [ 0.0000000e+00  2.0000000e+00  0.0000000e+00 -1.7890625e+02]
 [ 0.0000000e+00  0.0000000e+00  3.0000000e+00 -1.1025000e+03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
-152.0
neagtive value detecte

n4 processing part-2 started with 100199...
Reading Files.....
Reading Nifti Files.....
Affine:[[ 2.00000000e+00  0.00000000e+00  0.00000000e+00 -2.23593750e+02]
 [ 0.00000000e+00  2.00000000e+00  0.00000000e+00 -1.92701263e+02]
 [ 0.00000000e+00  0.00000000e+00  3.00000000e+00 -1.06770081e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]], Image Shape: (226, 184, 316)
Transforming Images to RAS.....
-160.0
neagtive value detected
saving directory: /mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/temp/NAKO/n4_corrected_2/vol/100199


In [7]:
for vol in file_paths.keys():
    try:
        if vol_to_check_list is not None and vol not in vol_to_check_list:
            continue
        print(f'n4-biasfield-correction starting with {vol}...')
        in_file = n4_dict[vol]['N4_2']['SCALED']
        opp_file = file_paths[vol]['ONE']['OPP']
        new_filename = opp_file.split('/')[-1].split('.')[0]
        output_file = f'{n4_corrected_data_dir}/vol/{vol}/{new_filename}_n4_corrected_sitk.nii.gz'
        SITK_N4_normalization(in_file, opp_file, output_file)
        n4_dict[vol]['N4_2']['OPP_CORRECTED'] = output_file
        file_paths[vol]['N4_2'] = n4_dict[vol]['N4_2']
    except Exception as e:
        print('ERROR:',e)
        continue

n4-biasfield-correction starting with 100000...
done
n4-biasfield-correction starting with 100001...
done
n4-biasfield-correction starting with 100002...
done
n4-biasfield-correction starting with 100003...
done
n4-biasfield-correction starting with 100004...
done
n4-biasfield-correction starting with 100005...
done
n4-biasfield-correction starting with 100007...
done
n4-biasfield-correction starting with 100010...
done
n4-biasfield-correction starting with 100011...
done
n4-biasfield-correction starting with 100012...
done
n4-biasfield-correction starting with 100014...
done
n4-biasfield-correction starting with 100015...
done
n4-biasfield-correction starting with 100016...
done
n4-biasfield-correction starting with 100018...
done
n4-biasfield-correction starting with 100019...
done
n4-biasfield-correction starting with 100021...
done
n4-biasfield-correction starting with 100022...
done
n4-biasfield-correction starting with 100023...
done
n4-biasfield-correction starting with 100024..

done
n4-biasfield-correction starting with 100172...
done
n4-biasfield-correction starting with 100173...
done
n4-biasfield-correction starting with 100174...
done
n4-biasfield-correction starting with 100175...
done
n4-biasfield-correction starting with 100176...
done
n4-biasfield-correction starting with 100177...
done
n4-biasfield-correction starting with 100178...
done
n4-biasfield-correction starting with 100179...
done
n4-biasfield-correction starting with 100180...
done
n4-biasfield-correction starting with 100182...
done
n4-biasfield-correction starting with 100184...
done
n4-biasfield-correction starting with 100185...
done
n4-biasfield-correction starting with 100186...
done
n4-biasfield-correction starting with 100187...
done
n4-biasfield-correction starting with 100188...
done
n4-biasfield-correction starting with 100189...
done
n4-biasfield-correction starting with 100190...
done
n4-biasfield-correction starting with 100191...
done
n4-biasfield-correction starting with 100

In [ ]:
def nako_vol_label_fix(vol, label):
    world_shape = np.max(np.array([list(vol.shape), list(label.shape)]), axis=0)
    final_label = np.zeros(tuple(world_shape))

    label_affine = label.affine
    vol_affine = vol.affine
    
    target_affine = vol_affine
    target_header = vol.header
    target_dim_v = vol.shape

    sx,sy,sz,ex,ey,ez = np.abs(get_points(label, vol))
    
    final_label[0:ex+sx, 0:ey+sy, sz:ez] = label.get_fdata()
    
    final_label = np.flip(final_label, axis=0)
    final_label = np.flip(final_label, axis=1)
    
    final_label_img = nb.Nifti1Image(final_label, target_affine, target_header)
    return vol, final_label_img

def nako_label_parts(label_parts, reference_labelmap=None):
    stitched_label = None
    mode = 'constant'
    order = 0
    if reference_labelmap is None:
        label_shape = get_freequent_shape([img.shape for img, _, _ in label_parts])
        reference_labelmap = [img for img, _, _ in label_parts if list(img.shape) == list(label_shape)][0]
    else:
        label_shape = reference_labelmap.shape

    stitched_label = np.zeros(label_shape)
    for labelmap_img, lidx, lname in label_parts:
        print(lidx, lname)
        labelmap_img = makeit_3d(labelmap_img)
        labelmap_img = resample_from_to(labelmap_img, [label_shape, reference_labelmap.affine], order=order, mode=mode, cval=0)
        
        sx,sy,sz,ex,ey,ez = np.abs(get_points(labelmap_img, reference_labelmap))
        
        labelmap = labelmap_img.get_fdata()
        labelmap = np.multiply(lidx, labelmap)
        stitched_label[0:ex+sx, 0:ey+sy, 0:ez+sz] += labelmap
        
        print("###############################################################################################") 
        
    labelmap = np.round(stitched_label)
    stitched_labeled_img = nb.Nifti1Image(labelmap, reference_labelmap.affine, reference_labelmap.header)
    
    return stitched_labeled_img

In [ ]:
print("STARTING NAKO LABEL-MAPS.")
print('Reading Label Maps.....')
for vol in file_paths.keys():
    print(vol)
    if vol_to_check_list is not None and vol not in vol_to_check_list:
        continue
    later = []
    print(file_paths[vol]['LABEL_PATHS'])
    if len(file_paths[vol]['LABEL_PATHS']) == 0:
        print(f"#################### ALERT:: NO LABELPATHS IN THE DICTIONARY FOR {vol} #########################")
        continue
    volume = nb.load(file_paths[vol]['N4_2']['OPP_CORRECTED'])
#     volume = nb.load(file_paths[vol]['ONE']['OPP'])
    img_ras_list = []
    for label_file_to_read in file_paths[vol]['LABEL_PATHS']:
        img_ras, lidx, labelname = read_ras(label_file_to_read, is_label=True)
        if labelname is None or img_ras is None:
            continue
        img_ras = makeit_3d(img_ras)
        img_ras = resample_to_output(img_ras, TARGET_RESOLUTION, order=0, mode='constant', cval=0)
        if labelname == 'SPLEEN':
            later.append([img_ras, lidx+LABEL_EXTENSION_FOR_OVERLAP_REMOVAL, labelname])
        else:
            img_ras_list.append([img_ras, lidx+LABEL_EXTENSION_FOR_OVERLAP_REMOVAL, labelname])
    
    img_ras_list.extend(later)
    
    s_label = nako_label_parts(img_ras_list)
    s_label = drop_overlapped_pixels(s_label, np.array(img_ras_list)[:, 1])
    volume, s_label = nako_vol_label_fix(volume, s_label)

    print('Viewing Stitched Images.....')
    volume_3_view_viewer(get_volume_data(volume))
    volume_3_view_viewer(get_volume_data(s_label))

    print('Saving Processed & Stitched Image.....')
    save_volume(volume, f'{processed_dir}/volume/{vol}')
    save_volume(s_label, f'{processed_dir}/label/{vol}')
    print('FINISHED.')